# RAG Pipeline with BioLLM (Pluggable Retrievers & Models)

In [28]:
#from huggingface_hub import snapshot_download
#snapshot_download(repo_id="microsoft/biogpt", local_dir="biogpt_local")

In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH = "/home/gulizhu/MDP/biogpt_local"   
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)

In [30]:

# === Config & Imports ===
import pandas as pd
import torch
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple

from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re, math
from collections import Counter, defaultdict

# Paths to your data files
CSV_PATH = Path("/home/gulizhu/MDP/combined_health_topics_with_source.csv")
TXT_PATH = Path("/home/gulizhu/MDP/textbook_pathology.txt")
XLSX_PATH = Path("/home/gulizhu/MDP/LLM Questions.xlsx")

# Model path (adjust to your BioLLM model)
MODEL_PATH = "/home/gulizhu/MDP/biogpt_local"  


## 1. Load Data (CSV + TXT + Excel QA)

In [31]:

# --- Load CSV (WHO topics) ---
df_csv = pd.read_csv(CSV_PATH)

# Use 'text' column as context
df_csv = df_csv.rename(columns={"text": "context"})
df_csv["source"] = "WHO"

# --- Load TXT (pathology textbook) and chunk ---
with open(TXT_PATH, "r", encoding="utf-8") as f:
    txt_content = f.read()

chunk_size = 800  # adjust as needed
txt_chunks = [txt_content[i:i+chunk_size] for i in range(0, len(txt_content), chunk_size)]
df_txt = pd.DataFrame([{"context": chunk, "source": "textbook_pathology"} for chunk in txt_chunks])

print("Textbook chunks:", len(df_txt))

# --- Load Excel QA ---
df_qa = pd.read_excel(XLSX_PATH)
df_qa = df_qa.rename(columns={c: c.lower() for c in df_qa.columns})

# normalize question column
if "question" not in df_qa.columns:
    if "q" in df_qa.columns:
        df_qa = df_qa.rename(columns={"q": "question"})
    elif "prompt" in df_qa.columns:
        df_qa = df_qa.rename(columns={"prompt": "question"})
    elif "ques" in df_qa.columns:
        df_qa = df_qa.rename(columns={"ques": "question"})
if "question" not in df_qa.columns:
    raise ValueError("Excel QA file must contain a question-like column")

# --- Combine knowledge sources ---
docs_df = pd.concat([df_csv[["context","source"]], df_txt], ignore_index=True)
print("Knowledge base size:", len(docs_df))
docs_df.head(2)


Textbook chunks: 4759
Knowledge base size: 6044


,context,source
0,Common goods for health are population-based f...,WHO
1,The social determinants of health (SDH) are th...,WHO


In [32]:
import pandas as pd

df_qa = pd.read_excel("LLM Questions.xlsx")
print(df_qa.columns)
print(df_qa.head())


Index(['question'], dtype='object')
                                            question
0  What is the role of a pathologist in cancer di...
1  Which biomarkers are key in the analysis of br...
2  How does a pathologist prepare and analyze a t...
3  What are key features that a pathologist looks...
4  What is immunohistochemistry and how is it use...


## 2. Define Retrievers (TF-IDF, BM25)

In [33]:

class TFIDFRetriever:
    def __init__(self, docs: List[str]):
        self.vectorizer = TfidfVectorizer(max_features=50000)
        self.doc_mat = self.vectorizer.fit_transform(docs)
        self.docs = docs

    def search(self, query: str, k=5):
        q_vec = self.vectorizer.transform([query])
        sims = cosine_similarity(q_vec, self.doc_mat)[0]
        idxs = sims.argsort()[::-1][:k]
        return [(int(i), float(sims[i])) for i in idxs]

class BM25Retriever:
    def __init__(self, docs: List[str], k1=1.5, b=0.75):
        self.docs = docs
        self.k1, self.b = k1, b
        self.tokenizer = re.compile(r"\w+").findall
        self.tokenized = [self.tokenizer(d.lower()) for d in docs]
        self.doc_lens = [len(t) for t in self.tokenized]
        self.avgdl = sum(self.doc_lens)/max(1,len(self.doc_lens))
        df = defaultdict(int)
        for toks in self.tokenized:
            for w in set(toks):
                df[w]+=1
        self.N = len(docs)
        self.idf = {w: math.log(1+(self.N-c+0.5)/(c+0.5)) for w,c in df.items()}
        self.tf = [Counter(toks) for toks in self.tokenized]

    def _score(self, q_toks, idx):
        score=0.0; dl=self.doc_lens[idx]; tf_d=self.tf[idx]
        for w in q_toks:
            if w not in self.idf: continue
            idf=self.idf[w]; f=tf_d.get(w,0)
            denom=f+self.k1*(1-self.b+self.b*dl/(self.avgdl or 1))
            score+=idf*(f*(self.k1+1))/(denom or 1e-12)
        return score

    def search(self, query:str,k=5):
        q_toks=self.tokenizer(query.lower())
        scores=[(i,self._score(q_toks,i)) for i in range(self.N)]
        scores.sort(key=lambda x:x[1], reverse=True)
        return scores[:k]


## 3. BioLLM Backend (swappable with other models)

In [34]:

@dataclass
class Message:
    role: str
    content: str

class BioLLMBackend:
    def __init__(self, model, tokenizer, device="cuda"):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device

    def generate(self, messages: List[Message]) -> str:
        query = next((m.content for m in messages[::-1] if m.role == "user"), "")
        context = "\n\n".join(m.content for m in messages if m.role in ("system", "tool"))
        context = context[:2000]
        prompt = f"Context:\n{context}\n\nQuestion:\n{query}\n\nAnswer:"
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=1024
        ).to(self.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=True,
                top_p=0.95,
                temperature=0.7
            )
        raw = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = raw.split("Answer:")[-1].strip() 
        return answer


## 4. RAG Pipeline

In [35]:

class SimpleRAG:
    def __init__(self, docs_df: pd.DataFrame, retriever="tfidf", llm=None):
        self.df = docs_df.reset_index(drop=True)
        self.contexts = self.df["context"].astype(str).tolist()
        if retriever=="tfidf":
            self.retriever = TFIDFRetriever(self.contexts)
        else:
            self.retriever = BM25Retriever(self.contexts)
        self.llm = llm

    def ask(self, query: str, k=3):
        hits = self.retriever.search(query, k)
        msgs=[Message(role="tool", content=self.contexts[i]) for i,_ in hits]
        msgs.append(Message(role="user", content=query))
        ans = self.llm.generate(msgs)
        return {
            "query": query,
            "context": " ".join(self.contexts[i][:500] for i,_ in hits),  
            "answer": ans,
            "hits": hits
        }



## 5. Initialize Model

In [36]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
llm = BioLLMBackend(model, tokenizer)


## 6. Compare Outputs Across Retrievers / Models

In [37]:

def compare_answers(df_qa: pd.DataFrame, retrievers=["tfidf","bm25"], llms=[("biogpt", llm)], n=5):
    sample = df_qa.sample(min(n, len(df_qa)), random_state=0)
    rows=[]
    for _,row in sample.iterrows():
        q = str(row["question"])
        for rname in retrievers:
            for lname, lbackend in llms:
                rag = SimpleRAG(docs_df, retriever=rname, llm=lbackend)
                out = rag.ask(q, k=3)
                rows.append({"question":q,"retriever":rname,"model":lname,"answer":out["answer"]})
    return pd.DataFrame(rows)

results = compare_answers(df_qa, retrievers=["tfidf","bm25"], llms=[("biollm", llm)], n=100)
results


,question,retriever,model,answer
0,How is pathology used in diagnosing soft tissu...,tfidf,biollm,"In this chapter, the authors have discussed th..."
1,How is pathology used in diagnosing soft tissu...,bm25,biollm,Is it appropriate to classify soft tissue sarc...
2,What is the importance of margins in pathology...,tfidf,biollm,The importance of margins is a matter of conje...
3,What is the importance of margins in pathology...,bm25,biollm,The margin of excision should be at least 2 cm...
4,Describe fluorescence in situ hybridization (F...,tfidf,biollm,Describe fluorescence in situ hybridisation (F...
...,...,...,...,...
105,What is the role of a pathologist in cancer di...,bm25,biollm,The pathologist may be able to play a role in ...
106,What are the differences between ductal and lo...,tfidf,biollm,The authors review the main histological featu...
107,What are the differences between ductal and lo...,bm25,biollm,Ductal carcinoma in situ (comedo and non-comed...
108,"What is the difference between sarcomas, carci...",tfidf,biollm,Are there any differences between the two grou...


In [38]:
results.to_csv("rag_results.csv", index=False, encoding="utf-8")

## Added: Embedding-based retrieval & comparison

In [39]:

# === Embedding backends config ===
EMBED_MODELS = [
    ("minilm", "sentence-transformers/all-MiniLM-L6-v2"),
    ("bge-small", "BAAI/bge-small-en-v1.5"),
]


In [40]:

import numpy as np

class EmbeddingBackend:
    def embed_texts(self, texts):
        raise NotImplementedError
    def embed_query(self, text):
        raise NotImplementedError

class SentenceTransformersEmbedding(EmbeddingBackend):
    def __init__(self, model_id: str, device: str = "cuda"):
        try:
            from sentence_transformers import SentenceTransformer
        except Exception as e:
            raise RuntimeError("sentence-transformers not installed. pip install sentence-transformers") from e
        self.model = SentenceTransformer(model_id, device=device)

    def embed_texts(self, texts):
        vecs = self.model.encode(texts, batch_size=64, show_progress_bar=False, convert_to_numpy=True, normalize_embeddings=True)
        return vecs

    def embed_query(self, text):
        return self.embed_texts([text])[0]

class HFMeanPoolingEmbedding(EmbeddingBackend):
    def __init__(self, model_id: str, device: str = "cuda"):
        from transformers import AutoModel, AutoTokenizer
        self.tok = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModel.from_pretrained(model_id).to(device)
        self.device = device

    def _mean_pool(self, outputs, attention_mask):
        last_hidden = outputs.last_hidden_state
        mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
        masked = last_hidden * mask
        summed = masked.sum(1)
        counts = mask.sum(1).clamp(min=1e-9)
        return (summed / counts).detach().cpu().numpy()

    def embed_texts(self, texts):
        import torch, numpy as _np
        all_vecs = []
        bs = 16
        for i in range(0, len(texts), bs):
            batch = texts[i:i+bs]
            enc = self.tok(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
            with torch.no_grad():
                out = self.model(**enc)
            vecs = self._mean_pool(out, enc["attention_mask"])
            vecs = vecs / (_np.linalg.norm(vecs, axis=1, keepdims=True) + 1e-9)
            all_vecs.append(vecs)
        return _np.vstack(all_vecs)

    def embed_query(self, text):
        return self.embed_texts([text])[0]

class EmbeddingRetriever:
    def __init__(self, docs, backend: EmbeddingBackend):
        self.docs = docs
        self.backend = backend
        self.doc_vecs = self.backend.embed_texts(docs)

    def search(self, query: str, k=5):
        q = self.backend.embed_query(query)
        sims = (self.doc_vecs @ q)
        idxs = np.argsort(-sims)[:k]
        return [(int(i), float(sims[i])) for i in idxs]


In [41]:

class SimpleRAG:
    def __init__(self, docs_df: pd.DataFrame, retriever="tfidf", llm=None, embed_backend: EmbeddingBackend = None):
        self.df = docs_df.reset_index(drop=True)
        self.contexts = self.df["context"].astype(str).tolist()
        self.llm = llm

        if retriever == "tfidf":
            self.retriever = TFIDFRetriever(self.contexts)
            self.retriever_name = "tfidf"
            self.embedding_name = "-"
        elif retriever == "bm25":
            self.retriever = BM25Retriever(self.contexts)
            self.retriever_name = "bm25"
            self.embedding_name = "-"
        elif retriever == "embed":
            if embed_backend is None:
                raise ValueError("embed_backend must be provided when retriever='embed'")
            self.retriever = EmbeddingRetriever(self.contexts, embed_backend)
            self.retriever_name = "embed"
            self.embedding_name = getattr(embed_backend, "model", getattr(embed_backend, "__class__", type(embed_backend))).__class__.__name__
        else:
            raise ValueError(f"Unknown retriever: {retriever}")

    def ask(self, query: str, k=3):
        hits = self.retriever.search(query, k)
        msgs = [Message(role="tool", content=self.contexts[i][:2000]) for i,_ in hits]
        msgs.append(Message(role="user", content=query))
        ans = self.llm.generate(msgs)
        combined_ctx = " ".join(self.contexts[i][:500] for i,_ in hits)
        return {"query":query, "answer":ans, "hits":hits, "context": combined_ctx}


In [42]:

def compare_answers(df_qa: pd.DataFrame, retrievers, llms, embed_models=None, n=5, device="cuda"):
    sample = df_qa.sample(min(n, len(df_qa)), random_state=0)
    rows = []

    def build_backend(model_id: str):
        try:
            return SentenceTransformersEmbedding(model_id, device=device)
        except Exception:
            return HFMeanPoolingEmbedding(model_id, device=device)

    for _, row in sample.iterrows():
        q = str(row["question"])
        for lname, lbackend in llms:
            for rname in retrievers:
                if rname == "embed":
                    if not embed_models:
                        continue
                    for embed_short, embed_id in embed_models:
                        backend = build_backend(embed_id)
                        rag = SimpleRAG(docs_df, retriever="embed", llm=lbackend, embed_backend=backend)
                        out = rag.ask(q, k=3)
                        rows.append({
                            "question": q,
                            "retriever": "embed",
                            "embedding": embed_short,
                            "model": lname,
                            "answer": out["answer"],
                            "context": out["context"]
                        })
                else:
                    rag = SimpleRAG(docs_df, retriever=rname, llm=lbackend)
                    out = rag.ask(q, k=3)
                    rows.append({
                        "question": q,
                        "retriever": rname,
                        "embedding": "-",
                        "model": lname,
                        "answer": out["answer"],
                        "context": out["context"]
                    })
    return pd.DataFrame(rows)


In [43]:

# Compare across TF-IDF, BM25, and Embedding retrievers; include embedding column
results = compare_answers(
    df_qa,
    retrievers=["tfidf", "bm25", "embed"],
    llms=[("biollm", llm)],
    embed_models=EMBED_MODELS,
    n=5
)
results

# Save
results.to_csv("rag_results_with_embeddings.csv", index=False, encoding="utf-8")
print("Saved -> rag_results_with_embeddings.csv")


Saved -> rag_results_with_embeddings.csv


## Hybrid retrieval (lexical shortlist + embedding re-rank)

In [44]:

class HybridRetriever:
    """Two-stage retriever: lexical (TF-IDF/BM25) -> embedding re-rank.
    base: 'tfidf' or 'bm25'
    top_m: candidate pool size from base retriever before re-ranking
    """
    def __init__(self, docs, base: str, embed_backend: EmbeddingBackend, top_m: int = 50):
        self.docs = docs
        self.top_m = top_m
        self.embed_backend = embed_backend

        if base == "tfidf":
            self.base = TFIDFRetriever(docs)
        elif base == "bm25":
            self.base = BM25Retriever(docs)
        else:
            raise ValueError("base must be 'tfidf' or 'bm25'")

        # Precompute doc embeddings once (can be large; adjust if memory is limited)
        self.doc_vecs = self.embed_backend.embed_texts(docs)

    def search(self, query: str, k=5):
        # Stage 1: lexical shortlist
        base_hits = self.base.search(query, k=self.top_m)  # [(idx, score)]
        cand_idxs = [i for i, _ in base_hits]

        # Stage 2: embedding re-rank within the shortlist
        q_vec = self.embed_backend.embed_query(query)
        import numpy as _np
        sims = (self.doc_vecs[cand_idxs] @ q_vec)  # cosine since we L2-normalized
        order = _np.argsort(-sims)[:k]
        reranked = [(int(cand_idxs[i]), float(sims[i])) for i in order]
        return reranked


In [45]:

class SimpleRAG:
    def __init__(self, docs_df: pd.DataFrame, retriever="tfidf", llm=None, embed_backend: EmbeddingBackend = None, hybrid_top_m: int = 50):
        self.df = docs_df.reset_index(drop=True)
        self.contexts = self.df["context"].astype(str).tolist()
        self.llm = llm

        # Pure lexical
        if retriever == "tfidf":
            self.retriever = TFIDFRetriever(self.contexts)
            self.retriever_name = "tfidf"
            self.embedding_name = "-"

        elif retriever == "bm25":
            self.retriever = BM25Retriever(self.contexts)
            self.retriever_name = "bm25"
            self.embedding_name = "-"

        # Pure embedding
        elif retriever == "embed":
            if embed_backend is None:
                raise ValueError("embed_backend must be provided when retriever='embed'")
            self.retriever = EmbeddingRetriever(self.contexts, embed_backend)
            self.retriever_name = "embed"
            self.embedding_name = getattr(embed_backend, "model", getattr(embed_backend, "__class__", type(embed_backend))).__class__.__name__

        # Hybrid: lexical shortlist + embedding re-rank
        elif retriever in ("hybrid_tfidf", "hybrid_bm25"):
            if embed_backend is None:
                raise ValueError("embed_backend must be provided for hybrid retrievers")
            base = "tfidf" if retriever == "hybrid_tfidf" else "bm25"
            self.retriever = HybridRetriever(self.contexts, base=base, embed_backend=embed_backend, top_m=hybrid_top_m)
            self.retriever_name = retriever
            self.embedding_name = getattr(embed_backend, "model", getattr(embed_backend, "__class__", type(embed_backend))).__class__.__name__

        else:
            raise ValueError(f"Unknown retriever: {retriever}")

    def ask(self, query: str, k=3):
        hits = self.retriever.search(query, k)
        # Truncate each passage to avoid overly long inputs
        msgs = [Message(role="tool", content=self.contexts[i][:2000]) for i,_ in hits]
        msgs.append(Message(role="user", content=query))
        ans = self.llm.generate(msgs)
        combined_ctx = " ".join(self.contexts[i][:500] for i,_ in hits)  # compact context for logging/export
        return {"query": query, "answer": ans, "hits": hits, "context": combined_ctx}


In [46]:

def compare_answers(df_qa: pd.DataFrame, retrievers, llms, embed_models=None, n=5, device="cuda", hybrid_top_m: int = 50):
    sample = df_qa.sample(min(n, len(df_qa)), random_state=0)
    rows = []

    def build_backend(model_id: str):
        try:
            return SentenceTransformersEmbedding(model_id, device=device)
        except Exception:
            return HFMeanPoolingEmbedding(model_id, device=device)

    for _, row in sample.iterrows():
        q = str(row["question"])

        for lname, lbackend in llms:
            for rname in retrievers:
                # retrievers that require embeddings
                if rname in ("embed", "hybrid_tfidf", "hybrid_bm25"):
                    if not embed_models:
                        continue
                    for embed_short, embed_id in embed_models:
                        backend = build_backend(embed_id)
                        rag = SimpleRAG(
                            docs_df,
                            retriever=rname,
                            llm=lbackend,
                            embed_backend=backend,
                            hybrid_top_m=hybrid_top_m
                        )
                        out = rag.ask(q, k=3)
                        rows.append({
                            "question": q,
                            "retriever": rname,
                            "embedding": embed_short,
                            "model": lname,
                            "answer": out["answer"],
                            "context": out["context"]
                        })
                else:
                    # lexical-only baselines
                    rag = SimpleRAG(docs_df, retriever=rname, llm=lbackend)
                    out = rag.ask(q, k=3)
                    rows.append({
                        "question": q,
                        "retriever": rname,
                        "embedding": "-",
                        "model": lname,
                        "answer": out["answer"],
                        "context": out["context"]
                    })

    return pd.DataFrame(rows)


In [47]:

# Evaluate combinations: lexical (tfidf/bm25), pure embedding, and hybrid (tfidf+embed, bm25+embed)
retrievers_to_test = ["tfidf", "bm25", "embed", "hybrid_tfidf", "hybrid_bm25"]
results = compare_answers(
    df_qa,
    retrievers=retrievers_to_test,
    llms=[("biollm", llm)],
    embed_models=EMBED_MODELS,
    n=5,
    hybrid_top_m=50
)
results

# Save outputs with both retriever and embedding columns
results.to_csv("rag_results_retriever_embedding_matrix.csv", index=False, encoding="utf-8")
print("Saved -> rag_results_retriever_embedding_matrix.csv")


Saved -> rag_results_retriever_embedding_matrix.csv


In [48]:
results

,question,retriever,embedding,model,answer,context
0,How is pathology used in diagnosing soft tissu...,tfidf,-,biollm,In this chapter we will discuss the various ty...,ectodermal tissues of the\nperipheral and auto...
1,How is pathology used in diagnosing soft tissu...,bm25,-,biollm,Do we need to be aware of the clinical and rad...,ectodermal tissues of the\nperipheral and auto...
2,How is pathology used in diagnosing soft tissu...,embed,minilm,biollm,"In this review article, we discuss the role of...","f tumours\nare also locally destructive, infil..."
3,How is pathology used in diagnosing soft tissu...,embed,bge-small,biollm,There are many soft tissue sarcomas that have ...,or\ntonofilaments or cell organelles.\n5.\nCyt...
4,How is pathology used in diagnosing soft tissu...,hybrid_tfidf,minilm,biollm,A.,"f tumours\nare also locally destructive, infil..."
5,How is pathology used in diagnosing soft tissu...,hybrid_tfidf,bge-small,biollm,What is the role of pathology in diagnosis and...,or\ntonofilaments or cell organelles.\n5.\nCyt...
6,How is pathology used in diagnosing soft tissu...,hybrid_bm25,minilm,biollm,There are some well-known facts that the patho...,"f tumours\nare also locally destructive, infil..."
7,How is pathology used in diagnosing soft tissu...,hybrid_bm25,bge-small,biollm,Cytogenetics is important for the diagnosis an...,or\ntonofilaments or cell organelles.\n5.\nCyt...
8,What is the importance of margins in pathology...,tfidf,-,biollm,The concept of margin analysis in pathology ha...,Wilms tumor is a rare type of kidney cancer. I...
9,What is the importance of margins in pathology...,bm25,-,biollm,A keloid or hypertrophic scar is a keloid or a...,phic or destroyed.\nThere are some differences...
